In [3]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
#from BCEmbedding.tools.langchain import BCERerank
from FlagEmbedding import FlagReranker
import pandas as pd
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain.retrievers import ContextualCompressionRetriever
import csv
import os
import time
import pickle
from zhipuai import ZhipuAI
import csv
import json

In [5]:
model_name = "BAAI/bge-large-zh-v1.5"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}
embedding = HuggingFaceBgeEmbeddings(
                model_name=model_name,
                model_kwargs=model_kwargs,
                encode_kwargs=encode_kwargs,
                query_instruction="为这个句子生成表示以用于检索相关文章:"
            )

In [ ]:
reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True) # Setting use_fp16 to True speeds up 
db = FAISS.load_local('tu_document_1024_512',embedding,allow_dangerous_deserialization = True)
retriever=db.as_retriever( search_kwargs={"k":9})

In [ ]:
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(
        model_name="GLM-4-PLus",
        openai_api_base="https://open.bigmodel.cn/api/paas/v4",
        openai_api_key="3138eb04ab176ee38855bc7bd5883868.RZfclCazuZkKXA1f",
        streaming=False,)

In [ ]:
def format_docs(docs):
    return "".join(doc.page_content for doc in docs)
rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt 
    | llm
    | StrOutputParser()
)
rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)
extracted_data = []

In [ ]:
with open('val.jsonl', 'r') as file:
    for line in file:
        data = json.loads(line)
        question = data['input_field']
        prompt="以下问题只关于TuGraph-DB\n"+ question +"\n要求：直接回答结果，如果不知道回答不知道，不要添加额外内容，不要进行推测。回答长度不要超过100字。"#提示词这块是需要工程化来做的
        result = rag_chain_with_source.invoke(question)
        extracted_info = {"id": data["id"],"output_field": result['answer']}
        extracted_data.append(extracted_info)
        print(question,result['answer'])
        print('\n')
#print(extracted_data)
# 将提取的数据写入到answer.jsonl文件中
with open('val_ans.jsonl', 'w',encoding='utf-8') as f:
    for item in extracted_data:
        # 将每个字典转换为JSON格式的字符串，并写入文件
        json_line = json.dumps(item,ensure_ascii=False) + '\n'
        f.write(json_line)
print("数据已写入到val_ans.jsonl文件中。")

In [ ]:
with open('test1.jsonl', 'r') as file:
    for line in file:
        data = json.loads(line)
        question = data['input_field']
        prompt="以下问题只关于TuGraph-DB\n"+ question +"\n要求：直接回答结果，如果不知道回答不知道，不要添加额外内容，不要进行推测。回答长度不要超过100字。"#提示词这块是需要工程化来做的
        result = rag_chain_with_source.invoke(question)
        extracted_info = {"id": data["id"],"output_field": result['answer']}
        extracted_data.append(extracted_info)
        print(question,result['answer'])
        print('\n')
#print(extracted_data)
# 将提取的数据写入到answer.jsonl文件中
with open('answer2.jsonl', 'w',encoding='utf-8') as f:
    for item in extracted_data:
        # 将每个字典转换为JSON格式的字符串，并写入文件
        json_line = json.dumps(item,ensure_ascii=False) + '\n'
        f.write(json_line)
print("数据已写入到answer.jsonl文件中。")